# PROBLEM STATEMENT
You will build a deep learning model to predict the future stock prices of a company using historical stock price data. The goal is to predict the next 10 days of stock prices using a sequence of the past 60 days of price

1. Problem Definition
2. Data Collection
3. Data Preprocessing
4. Model Building (RNN, LSTM, GRU)
5. Model Training
6. Model Evaluation
7. Model Prediction
8. Model Deployment (Optional)

#1. Problem Definition

*   Objective: Predict the next 10 days of stock prices based on the previous 60 days' stock prices.

*  Target Variable: Stock closing price. Type: Regression problem (continuous value prediction).




##2. DATA COLLECTION


In [1]:
!pip install yfinance
import yfinance as yf



In [2]:
# Download Google stock data for 5 years
data = yf.download('GOOG', start='2018-01-01', end='2023-01-01')
data = data[['Close']]  # Use only closing prices
data.head()

[*********************100%***********************]  1 of 1 completed


,Close
Date,
2018-01-02,53.250000
2018-01-03,54.124001
2018-01-04,54.320000
2018-01-05,55.111500
2018-01-08,55.347000


## 3. Data Preprocessing
Preprocess the data for input into the RNN model.

3.1. Data Scaling
Neural networks work better when the input data is scaled, so we'll use MinMaxScaler to scale the stock prices between 0 and 1.

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Prepare the training and testing datasets (80% training, 20% testing)
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]


Data Preparation (Time-Series)
Create sequences of 60 days of prices to predict the next day’s price.

In [ ]:
def create_sequences(data, sequence_length=60):
    sequences = []
    labels = []
    for i in range(sequence_length, len(data)):
        sequences.append(data[i-sequence_length:i])
        labels.append(data[i])
    return np.array(sequences), np.array(labels)

sequence_length = 60
X_train, y_train = create_sequences(train_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)


NameError: name 'train_data' is not defined

#4. Model Building (RNN, LSTM, GRU)
4.1. Building a Simple RNN Model
We'll start with a simple RNN, then modify it using LSTM and GRU.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Dropout

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(SimpleRNN(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

4.2. LSTM Model (Improved RNN)


In [ ]:
# Define the LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_lstm.add(Dropout(0.2))
model_lstm.add(LSTM(50, return_sequences=False))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(25))
model_lstm.add(Dense(1))

model_lstm.compile(optimizer='adam', loss='mean_squared_error')

4.3. GRU Model (Another RNN Variant)


In [ ]:
# Define the GRU model
model_gru = Sequential()
model_gru.add(GRU(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_gru.add(Dropout(0.2))
model_gru.add(GRU(50, return_sequences=False))
model_gru.add(Dropout(0.2))
model_gru.add(Dense(25))
model_gru.add(Dense(1))

model_gru.compile(optimizer='adam', loss='mean_squared_error')

#5. Model Training
Train the model for a number of epochs and monitor the loss.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Function to create and compile models
def create_model(model_type, input_shape):
    model = Sequential()

    if model_type == 'RNN':
        model.add(SimpleRNN(50, return_sequences=True, input_shape=input_shape))
        model.add(Dropout(0.2))
        model.add(SimpleRNN(50, return_sequences=False))

    elif model_type == 'LSTM':
        model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
        model.add(Dropout(0.2))
        model.add(LSTM(50, return_sequences=False))

    elif model_type == 'GRU':
        model.add(GRU(50, return_sequences=True, input_shape=input_shape))
        model.add(Dropout(0.2))
        model.add(GRU(50, return_sequences=False))

    model.add(Dropout(0.2))
    model.add(Dense(25))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


# Initialize results storage
results = []

# Train and evaluate models (RNN, LSTM, GRU)
for model_type in ['RNN', 'LSTM', 'GRU']:
    print(f"\nTraining {model_type} Model")

    model = create_model(model_type, (X_train_reshaped.shape[1], 1))

    # Train the model
    model.fit(X_train_reshaped, y_train, epochs=10, batch_size=64, validation_data=(X_test_reshaped, y_test))


#6. Model Evaluation
Evaluate the model using Mean Squared Error (MSE) or Mean Absolute Error (MAE).

In [ ]:
# Evaluate the model
    y_pred = model.predict(X_test_reshaped)
    y_pred_scaled = scaler.inverse_transform(y_pred)
    y_test_scaled = scaler.inverse_transform(y_test.reshape(-1, 1))

    mse = mean_squared_error(y_test_scaled, y_pred_scaled)
    mae = mean_absolute_error(y_test_scaled, y_pred_scaled)

    results.append([model_type, mse, mae])

    # Plot actual vs predicted
    plt.figure(figsize=(10, 6))
    plt.plot(y_test_scaled, label="Actual Prices")
    plt.plot(y_pred_scaled, label=f"Predicted Prices ({model_type})")
    plt.title(f"Actual vs Predicted Prices ({model_type})")
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

# Display results in a table
results_df = pd.DataFrame(results, columns=['Model Type', 'Mean Squared Error', 'Mean Absolute Error'])
print("\nModel Evaluation Results:")
print(results_df)

#Additional Enhancements:
1. Add Multiple Inputs: Use not only closing prices but also features like volume, open, high, and low prices for improved predictions.
2. Hyperparameter Tuning: Experiment with different learning rates, batch sizes, and the number of layers to optimize the model.
3. Ensemble

 # CONCLUSION
###pipeline provides an end-to-end comparison of RNN, LSTM, and GRU models on a time-series prediction task. The tabular results and visual plots will help you compare which model performs better for stock price forecasting.